In [1]:
import orjson as json
import numpy as np
import pandas as pd
import tqdm
from mappings import slot_hero
from features import objectives_features

In [2]:
for i, line in enumerate(open('../data/train_matches.jsonl')):
    game = json.loads(line)
    print(game['objectives'])
    if i > 5:
        break

[{'time': 124.203, 'type': 'CHAT_MESSAGE_FIRSTBLOOD', 'slot': 8, 'key': 2, 'player_slot': 131}]
[{'time': 2.701, 'type': 'CHAT_MESSAGE_FIRSTBLOOD', 'slot': 6, 'key': 2, 'player_slot': 129}, {'time': 579.685, 'type': 'CHAT_MESSAGE_TOWER_KILL', 'team': 2, 'slot': 4, 'player_slot': 4}, {'time': 652.601, 'type': 'CHAT_MESSAGE_TOWER_KILL', 'team': 2, 'slot': -1}]
[]
[{'time': 154.262, 'type': 'CHAT_MESSAGE_FIRSTBLOOD', 'slot': 2, 'key': 6, 'player_slot': 2}]
[{'time': -10.997, 'type': 'CHAT_MESSAGE_FIRSTBLOOD', 'slot': 1, 'key': 9, 'player_slot': 1}]
[]
[{'time': 78.647, 'type': 'CHAT_MESSAGE_FIRSTBLOOD', 'slot': 4, 'key': 8, 'player_slot': 4}]


In [3]:
objective_types = []
team = []
player = []
for i, line in enumerate(open('../data/train_matches.jsonl')):
    game = json.loads(line)
    cnt_aegis = 0
    for obj in game['objectives']:
        objective_types.append(obj['type'])
        team.append(obj.get('team', None))
        player.append(slot_hero[obj['player_slot']] if 'player_slot' in obj else None)
        if obj['type'] == 'CHAT_MESSAGE_AEGIS':
            cnt_aegis += 1
    if cnt_aegis > 1:
        print("# of aegis events:", cnt_aegis, game['objectives'])
        break

# of aegis events: 2 [{'time': 17.862, 'type': 'CHAT_MESSAGE_FIRSTBLOOD', 'slot': 8, 'key': 1, 'player_slot': 131}, {'time': 573.593, 'type': 'CHAT_MESSAGE_TOWER_KILL', 'team': 2, 'slot': 2, 'player_slot': 2}, {'time': 828.531, 'type': 'CHAT_MESSAGE_TOWER_KILL', 'team': 3, 'slot': 8, 'player_slot': 131}, {'time': 967.63, 'type': 'CHAT_MESSAGE_TOWER_KILL', 'team': 2, 'slot': 2, 'player_slot': 2}, {'time': 1010.353, 'type': 'CHAT_MESSAGE_TOWER_KILL', 'team': 3, 'slot': 8, 'player_slot': 131}, {'time': 1010.487, 'type': 'CHAT_MESSAGE_TOWER_KILL', 'team': 2, 'slot': -1}, {'time': 1186.577, 'type': 'CHAT_MESSAGE_TOWER_KILL', 'team': 2, 'slot': -1}, {'time': 1191.043, 'type': 'CHAT_MESSAGE_TOWER_KILL', 'team': 3, 'slot': 8, 'player_slot': 131}, {'time': 1281.42, 'type': 'CHAT_MESSAGE_ROSHAN_KILL', 'team': 2}, {'time': 1281.62, 'type': 'CHAT_MESSAGE_AEGIS', 'slot': 2, 'player_slot': 2}, {'time': 1382.196, 'type': 'CHAT_MESSAGE_TOWER_KILL', 'team': 2, 'slot': 2, 'player_slot': 2}, {'time': 164

In [4]:
for i, line in enumerate(open('../data/test_matches.jsonl')):
    game = json.loads(line)
    for obj in game['objectives']:
        objective_types.append(obj['type'])
        team.append(obj.get('team', None))
        player.append(slot_hero[obj['player_slot']] if 'player_slot' in obj else None)
        break
    break

In [5]:
df = pd.DataFrame(np.array([objective_types, team, player]).T, columns=['objective_type', 'team', 'player'])

In [6]:
df.head()

,objective_type,team,player
0,CHAT_MESSAGE_FIRSTBLOOD,None,d4
1,CHAT_MESSAGE_FIRSTBLOOD,None,d2
2,CHAT_MESSAGE_TOWER_KILL,2,r5
3,CHAT_MESSAGE_TOWER_KILL,2,None
4,CHAT_MESSAGE_FIRSTBLOOD,None,r3


In [7]:
# ~ 4k games don't have firstblood at the time of data colection
df.groupby('objective_type').size()

objective_type
CHAT_MESSAGE_AEGIS             2
CHAT_MESSAGE_BARRACKS_KILL     2
CHAT_MESSAGE_FIRSTBLOOD       15
CHAT_MESSAGE_ROSHAN_KILL       2
CHAT_MESSAGE_TOWER_KILL       55
dtype: int64

In [8]:
obj.keys()

dict_keys(['time', 'type', 'team', 'slot', 'player_slot'])

In [9]:
game['objectives']

[]

In [10]:
train_objectives = []
for i, line in tqdm.tqdm(enumerate(open('../data/train_matches.jsonl'))):
    game = json.loads(line)
    train_objectives.append(objectives_features(game))

39675it [01:12, 546.70it/s]


In [11]:
test_objectives = []
for i, line in tqdm.tqdm(enumerate(open('../data/test_matches.jsonl'))):
    game = json.loads(line)
    test_objectives.append(objectives_features(game))

10000it [00:18, 552.89it/s]


In [12]:
train_features = pd.DataFrame(train_objectives)
test_features = pd.DataFrame(test_objectives)

In [13]:
train_features.to_csv('../features/objectives_train.csv', index=False)
test_features.to_csv('../features/objectives_test.csv', index=False)